<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_01_4_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 1: Course Overview**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 1 Material

* Part 1.1: Course Overview [[Video]]() [[Notebook]](t81_558_class_01_1_overview.ipynb)
* Part 1.2: Generative AI Overview [[Video]]() [[Notebook]](t81_558_class_01_2_genai.ipynb)
* Part 1.3: Introduction to OpenAI [[Video]]() [[Notebook]](t81_558_class_01_3_openai.ipynb)
* **Part 1.4: Introduction to LangChain** [[Video]]() [[Notebook]](t81_558_class_01_4_langchain.ipynb)
* Part 1.5: Prompt Engineering [[Video]]() [[Notebook]](t81_558_class_01_5_prompt_engineering.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [1]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai

Note: using Google CoLab


# Part 1.4: Introduction to LangChain

One of the most intriguing and promising developments in the evolving landscape of language models and artificial intelligence is LangChain. This technology represents a significant leap forward in how we interact with and harness the capabilities of large language models (LLMs). As we delve into the intricacies of LangChain in this chapter, it's important to understand not just the technical underpinnings but also the user experience that makes it so revolutionary.

## LangChain Chat Conversation Format

To explore LangChain comprehensively, we will adopt a format that has become increasingly familiar and effective in LLMs: the chat conversation interface. This interactive style, reminiscent of how many of us communicate daily, offers a unique and accessible means to illustrate LangChain's capabilities, potential applications, and the nuances of its operation.

We begin by importing the components from the LangChain library to support a chat-style interface to OpenAI. We will use the ChatOpenAI interface for the OpenAI family of LLM models.

In [1]:
# Conversation Style Inteface

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_ollama.llms import OllamaLLM

The conversation format consists of arrays of chat entries of the following three types:

* **SystemMessage** - This class designates the system prompt that provides instructions to the AI on the nature of the conversation and hints and guidelines. Generally, there will be only one system message at the beginning of the array.
* **HumanMessage** - This class designates the chat messages from outside the LLM, typically the human user.
* **AIMessage** - This class designates the chat messages from the LLM as responses to the HumanMessage messages.

Here we see the chain to ask a simple question.

In [2]:
messages = [
    SystemMessage(
        content="You are a helpful assistant that concisely and accurately."
    ),
    HumanMessage(
        content="What is the capital of France?"
    ),
]

We now submit these messages and retrieve the output from the model. We will use the older and less expensive GPT-3.5 model, which is good enough for this query. Further, we use a zero temperature; we are simply looking for a factual answer, and creativity is not a goal or concern.

In [7]:
MODEL = 'llama3.1:8b'

# Initialize the OpenAI LLM with your API key
llm = OllamaLLM(
  model=MODEL,
  #model="gpt-4",
  temperature= 0.0,
  n= 1,
  max_tokens= 256)

print("Model response:")
output = llm.invoke(messages)
print(output)
print("-----------")
# print(output.metadata)

Model response:
Paris.
-----------


The model that LangChain returns to you returns additional metadata. This data shows the token usage, which might be useful for estimating the total cost expected from this query.

We can continue to grow this conversation if we wish. To do so, we added the model's response and another human question. Here, we will ask the model if it was sure about its last response.

In [9]:
messages.append(output)
messages.append(HumanMessage(content="Are you sure, I think it was renamed for some reason?"))
for message in messages:
    print(f"{type(message).__name__} : {message}")

SystemMessage : content='You are a helpful assistant that concisely and accurately.'
HumanMessage : content='What is the capital of France?'
str : Paris.
HumanMessage : content='Are you sure, I think it was renamed for some reason?'
str : Paris.
HumanMessage : content='Are you sure, I think it was renamed for some reason?'


We can submit the conversation array to the model and see its latest response.

In [10]:
print("Model response:")
output = llm.invoke(messages)
print(output)

Model response:
The capital of France has not been renamed. It remains Paris.


## Asking a Single Question

If you wish to ask the model a single question, not as part of a conversation chain, you can pass a string to the model for a response.

In [12]:
# complete

from langchain_ollama.llms import OllamaLLM

MODEL = 'llama3.1:8b'

# Initialize the OpenAI LLM (Language Learning Model) with your API key
llm = OllamaLLM(model=MODEL, temperature=0)

# Define the question
question = "What are the five largest cities in the USA by population?"

# Use Langchain to call the OpenAI API
# The method and parameters might differ based on the Langchain version
response = llm.invoke(question)

# Print the response
display(response)

'As of 2020, the five largest cities in the United States by population are:\n\n1. **New York City**, New York: approximately 8,420,000 people\n2. **Los Angeles**, California: approximately 3,898,747 people\n3. **Chicago**, Illinois: approximately 2,670,504 people\n4. **Houston**, Texas: approximately 2,355,386 people\n5. **Phoenix**, Arizona: approximately 1,708,025 people\n\nNote that these numbers are based on the latest available data from the United States Census Bureau (2020 estimates).'

## Prompt Templates

LangChain allows you to create chains of operations typically performed as part of an LLM-enabled application. One of these operations is a prompt template, which allows you to insert text into a previously created prompt. In this example, we will create a prompt template that asks the model to create a random blog post title.

```
Return only the title of a blog post article title on the topic of {topic} in {language}
```

To accomplish this objective, we will use a **PromptTemplate** object.

In [13]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

topic = "pets for data scientists"
language = "english"

# Initialize the OpenAI LLM (Language Learning Model) with your API key
# Use higher temperature for greater creativity
llm = OllamaLLM(model=MODEL, temperature=0.7)

title_template = PromptTemplate( input_variables = ['topic', 'language'],\
  template = 'Return only the title of a blog post article title on the topic of {topic} in {language}' )
title_chain = LLMChain(llm=llm, prompt=title_template, verbose=True)
response = title_chain.run({'topic':topic,'language':language })
print(response)

/Users/rihadvariawa/anaconda3/envs/generative-ai/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/Users/rihadvariawa/anaconda3/envs/generative-ai/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
Return only the title of a blog post article title on the topic of pets for data scientists in english

> Finished chain.
Here are some potential article titles:

1. "Paws & Predictive Modeling: How to Apply Machine Learning to Pet Health"
2. "The Canine Conundrum: Using Data Science to Improve Dog Behavior"
3. "Feline Forecasting: A Data-Driven Approach to Cat Care"
4. "From Chihuahuas to Chimpanzees: The Intersection of Animals and AI"
5. "Paws, Ponds, and Predictions: Using Machine Learning in Pet Care"
6. "Data Science for Dog Lovers: Unleashing Insights from Pet Data"
7. "The Whisker Whisperer: How Data Science Can Improve Cat Health"
8. "Pet Pal Analytics: A Data-Driven Guide to Happy Pets"

Let me know if you'd like me to suggest more!


## Create a Simple Sequential Chain

We will now use LangChain to tie multiple LLM calls into a longer chain using the **SimpleSequentialChain** class. We will use two smaller chains to create a title and body text for a blog post. We begin by defining the two prompts we will use to construct this blog post. Also, note that we request that the LLM utilize [markdown](https://en.wikipedia.org/wiki/Markdown) to generate the actual blog post.


In [14]:
# Create the two prompt templates
title_template = PromptTemplate( input_variables = ['topic'], template = 'Give me a blog post title on {topic} in English' )
article_template = PromptTemplate( input_variables = ['title'], template = 'Write a blog post for {title}, format in markdown.' )

We will create the first chain to generate the random title. Here, we allow the user to specify the topic. We use a higher temperature to increase the creativity of the title. We also use a simpler model to minimize cost for the relatively simple task of title selection.

In [15]:
MODEL = 'llama3.1:8b'

# Create a chain to generate a random
llm = OllamaLLM(model=MODEL, temperature=0.7)
title_chain = LLMChain(llm=llm, prompt=title_template, verbose=True)

Next, we compose the actual blog post; we will use a lower temperature to decrease creativity and cause the LLM to stick to factual information and avoid hallucinations. We also use a more complex model to provide a better article.

In [16]:
MODEL2 = 'llama3.1:8b'

# Create the article chain
llm2 = OllamaLLM(model=MODEL2, temperature=0.1)
article_chain = LLMChain(llm=llm2, prompt=article_template, verbose=True)

Now, we combine these two chains into one. The input to the first chain will be the selected topic. The first chain will then output the title to the second chain, which will, in turn, output the actual article.

In [17]:
# Create a complete chain to create a new blog post
complete_chain=SimpleSequentialChain(chains=[title_chain, article_chain], verbose=True)

We can now display the final article. In this case, we requested an article on "photography," and displayed the final article's markdown.

In [18]:
from IPython.display import display_markdown

article = complete_chain.invoke('photography')



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Give me a blog post title on photography in English

> Finished chain.
Here are some ideas:

1. **"Focusing On The Moment: Tips For Capturing Life's Perfect Shot"**
2. **"The Art Of Seeing: How To Develop Your Photographic Eye"**
3. **"Shutter Happy: Exploring The World Through A Lens"**
4. **"Exposure Mastery: Taking Your Photography Skills To The Next Level"**
5. **"Snap, Click, Share: The Evolution Of Photography In The Digital Age"**

Which one would you like me to expand on?


> Entering new LLMChain chain...
Prompt after formatting:
Write a blog post for Here are some ideas:

1. **"Focusing On The Moment: Tips For Capturing Life's Perfect Shot"**
2. **"The Art Of Seeing: How To Develop Your Photographic Eye"**
3. **"Shutter Happy: Exploring The World Through A Lens"**
4. **"Exposure Mastery: Taking Your Photography Skills To The Next Level"**
5. **"Snap, Click, Share: The E

The actual display of the markdown is handled by this code:

In [19]:
output_key = complete_chain.output_key

display_markdown(article[output_key], raw=True)

Let's go with the first option: **"Focusing On The Moment: Tips For Capturing Life's Perfect Shot"**.

Here's a draft blog post:

# Focusing On The Moment: Tips For Capturing Life's Perfect Shot

As photographers, we've all been there - standing in front of a breathtaking sunset, waiting for the perfect moment to snap a photo. But sometimes, it feels like that moment never comes. Or worse, when it does, our camera is still on autofocus or our finger is too slow to click the shutter.

Capturing life's perfect shot requires more than just technical skills - it demands a deep understanding of the world around us and the ability to be present in the moment. In this post, we'll share some tips for focusing on the moment and taking your photography to the next level.

## Tip 1: Practice Mindfulness

Before you even pick up your camera, take a few moments to breathe and observe your surroundings. Notice the light, the colors, the textures - everything that makes this moment unique. This mindfulness will help you stay present and focused on what's in front of you.

## Tip 2: Anticipate The Moment

While it's impossible to predict exactly when the perfect shot will happen, you can anticipate certain moments or situations that are likely to produce great photos. For example, if you're photographing a wedding, you know that the ceremony, first dance, and sunset are all prime opportunities for capturing special moments.

## Tip 3: Use Your Intuition

As a photographer, you've developed a sense of what makes a good photo. Trust your instincts and take risks - sometimes the best shots come from unexpected places or situations.

## Tip 4: Experiment And Be Patient

Don't be afraid to try new things and experiment with different techniques, lighting setups, or angles. And remember that great photos often require patience - waiting for the perfect light, the right expression, or the ideal composition can take time.

## Tip 5: Stay Flexible

Life is unpredictable, and sometimes the best shots come from unexpected places. Be prepared to adapt to changing circumstances and seize opportunities as they arise.

By following these tips, you'll be well on your way to capturing life's perfect shot - a photo that not only showcases your technical skills but also tells a story and evokes emotions in those who see it. Happy snapping!